# Building Custom Images Through Docker Server

## Creating Docker Images

So far in the course, we have ran Docker commands that are created by other developers. 
- <img src="./images/docker_11.png" alt="Drawing" style="width: 300px;"/>

**To create:**
1. Create a Dockerfile
    - Configuration to define how our container should behave
2. Docker Client
3. Docker Server
    - The Docker Server will use the Dockerfile and use the lines of configuration that have inside of it and then build a useable image that can then be used to start a new container
4. Usable image!


**Creating a Dockerfile**
1. Specify a base image
2. Run some commands to install additional programs
3. Specify a command to run on container startup

## Building a Dockerfile

**The goal is to create an image that runs redis-server**

In the file: `"/Users/alexguanga/All_Projects/exponential_learning/Udemy/DockerKubernetesCompleteGuide/projects/redis-image"`, use the Dockerfile
- The Dockerfile should output commands in the terminal (including the id)


## Docker Teardown

If we use the Dockerfile above (directory: `"/Users/alexguanga/All_Projects/exponential_learning/Udemy/DockerKubernetesCompleteGuide/projects/redis-image"`); below are the commands:

Below are the instructions from the Dockerfile, which tells the Docker Server to do some very specific preparation steps on the image.
1. FROM
    - Use to specify the Docker Image that we want to use as a base 
2. RUN
    - Use to execute some command while we are preparing our custom image
3. CMD
    - The command instructions specifies what we should execute when our Docker Image is used to start a new container 

## What's a Base Image?

Oddly enough, the instructors provides the following analogy:
- Writing a Dockerfile == Being given a computer with no Operating System (OS) and being told to install Chrome
    - Keep in mind that in order to even install Chrome, we need an operating system... there's no other way to do so!


**Base Image**
<img src="./images/docker_12.png" alt="Drawing" style="width: 300px;"/>
- In the screenshot above, notice how the steps are very similar to installing Chrome.
- Before you can even install Chrome, you need to specify a base image (operating system)
    - `FROM alpine`; the command used in the Dockerfile
        - But why alpine?
        - Preference and compatabilility. We use operating systems that can fulfill our tasks.
        - Alpline in specific is being used because it includes a default set of programs that are very useful for creating a Dockerfile
    - `RUN apk add --update redis`
        - Using the Chrome analogy, we have certain steps that must correctly implemented before we install Chrome. 
        - Thus, the command above is able to install the requried steps to use redis!

## The Build Process in Detail

**`docker build .`**
- The `build` instructs Docker to tak a Dockerfile and generate an image from it.
- The `.` is specifying what is called the **build context**. The build content is the set of files and folders that belong to our project.
- It is the set of files and folders that we want to encapsulate or wrap in this container


Process:
1. `FROM alpine`
    - First, it looks if the Base Image is located in the local machine. If not, it downloads the Base Image from Docker Hub. 
    - There are outputs that specify that the Base Image was appropriate downloaded.
2. `RUN apk add --update redis`
    - First, there's an ID in the first output from the command above "Running in ...". Notice that a few lines below the first output, the output states "Removing intermediate container ...". It's the same ID for both outputs.
3. `CMD ["redis-server"]`
    - This step follows a similar procedure to step 2 where a "Running in ..." and "Removing intermediate container..." is outputted.



Explanation: "Running in" & "Removing intermediate..."
- `FROM alpine` & `RUN apk add --update redis`
    - As explained above, the outputs that occur for step 2 and step 3 is a bit strange... (read below to clarify confusion)
    - <img src="./images/docker_13.png" alt="Drawing" style="width: 300px;"/>
    - When the `RUN` command occurs, it backtracks to the previous command. In our case, it's the `FROM` command. Thus, it creates a container with the "alphine" Base Image. In the "alphine" Base Image, we have the File System (FS) Snapshot
    - The `RUN` commands get executed in the Container to run and thus, we should expect new files in the Container's Hardrive (i.e. redis)
    - After creating the Container, we remove the Container but we take a system snapshot and we save it as a temporary Image with an ID. 
    
    
- `RUN apk add --update redis` & `CMD ["redis-server"]`
    - <img src="./images/docker_14.png" alt="Drawing" style="width: 300px;"/> 
    - Similar to the previous step, the `CMD` command reverts back one step and thus, used the Base Image that was created in the `RUN` command.
    - A slight difference is that it never runs the command "redis-server" but if the Base Image is ever runned, it will use the "redis-server" command!


- At the end, the Base Image...
    - <img src="./images/docker_15.png" alt="Drawing" style="width: 300px;"/>

## A Brief Recap

Below is a diagram of the steps from the previous section
1. <img src="./images/docker_16.png" alt="Drawing" style="width: 300px;"/>
2. <img src="./images/docker_17.png" alt="Drawing" style="width: 300px;"/>
3. <img src="./images/docker_18.png" alt="Drawing" style="width: 300px;"/>

## Rebuilds with Cache

```
FROM alpine

RUN apk add --update redis
RUN apk add --update gcc

CMD ["redis-server"]

```


When we create a temporary container but take s snapshot of the File System and thus, we store the Image that was created at each of the step.

If we include an additional step: `RUN apk add --update gcc`
- For the commands that were previously ran, it no longer fetches Docker Hub for the installation of the packages as it was already downloaded it from Docker Hub.
    - There's a message `Using cache`. Therefore, from the previous time we ran the Dockerfile, it realizes that that step 1 and step 2 are the same and thus, step 2 can be ran as it was previously.
- For the new command, it runs the same steps as it normally would when it's a new Dockerfile being runned.


**Note**
- The order of the instructions mattter! If the instructions are changed, it can no longer use the cache and thus, it must go through the installation process again.

## Tagging an Image

Once we build the Image, we would the ID of the image when we want to run the Image to create a Container

However, using the ID to run the Image to create a Container is not ideal... 
- Tagging an Image: `docker build -t alexguanga/redis:latest .`
    - `-t alexguanga/redis:latest` tags the image
    - `.` specifies the directory of the files/folders to use for the build
- Tagging Flag: `-t alexguanga/redis:latest`
    - `alexguanga` is the Docker ID
    - `redis` is the Repo/Project name (could be anything)
    - `latest` is the Version


To use Tagging:
- `docker build -t alexguanga/redis:latest .`
- `docker run alexguanga/redis`

## Manual Image Generation with Docker Commit

While we can create a Container from an Image, we can also create a Image from a Container.

- `docker run -it alphine sh` #Start up a shell
- In the shell, explicit state `apk add --update redis`
- In a new terminal window, `docker ps`
- Then, `docker commit -c 'CMD ["redis-server"]' <id>`
- This would then print out an SHA-id... which could be used to create a new container (it's the Base Image created by the commands above)